In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath = os.path.sep.join([r'E:\FACE_MASK_DETECTION\MASK_NO_MASK\faceDetection',
                                 'deploy.prototxt'])
weightPath = os.path.sep.join([r'E:\FACE_MASK_DETECTION\MASK_NO_MASK\faceDetection',
                               'res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
net=cv2.dnn.readNet(prototxtPath,weightPath)

In [4]:
model = load_model(r'E:\FACE_MASK_DETECTION\MASK_NO_MASK\mask_no_mask.model')

In [55]:
img = cv2.imread(r'E:\FACE_MASK_DETECTION\MASK_NO_MASK\examples\ex4.jpg')
img = cv2.resize(img,(224,224))

In [56]:
(h,w) = img.shape[:2]

In [57]:
(h,w)

(224, 224)

In [58]:
blob = cv2.dnn.blobFromImage(img,1.0,(300,300),(104.0,177.0,123.0))


In [59]:
blob

array([[[[ -84.,  -83.,  -82., ...,  103.,   95.,   87.],
         [ -84.,  -83.,  -83., ...,  103.,   95.,   88.],
         [ -83.,  -83.,  -83., ...,  103.,   95.,   88.],
         ...,
         [ -75.,  -75.,  -75., ...,  124.,  125.,  123.],
         [ -75.,  -75.,  -75., ...,  122.,  123.,  123.],
         [ -74.,  -75.,  -75., ...,  121.,  122.,  122.]],

        [[-163., -163., -163., ...,   16.,    8.,    0.],
         [-163., -163., -163., ...,   16.,    8.,    0.],
         [-163., -164., -163., ...,   17.,    8.,    1.],
         ...,
         [-156., -157., -157., ...,   40.,   41.,   39.],
         [-156., -157., -157., ...,   38.,   39.,   38.],
         [-155., -156., -158., ...,   37.,   38.,   38.]],

        [[-122., -122., -122., ...,   71.,   63.,   56.],
         [-122., -123., -123., ...,   71.,   63.,   56.],
         [-122., -123., -123., ...,   71.,   63.,   56.],
         ...,
         [-119., -118., -117., ...,   97.,   97.,   95.],
         [-119., -118., -1

In [60]:
blob.shape

(1, 3, 300, 300)

In [61]:
net.setInput(blob)
detections = net.forward()

In [62]:
for i in range(0,detections.shape[2]):
    confidence = detections[0,0,i,2]
    
    if confidence > 0.5:
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY) = box.astype('int')
        
        (startX,startY) = (max(0,startX),max(0,startY))
        (endX,endY) = (min(w-1,endX), min(h-1,endY))
        
        face=img[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label == 'Mask' else (0,0,255)
        
        label = "{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        cv2.putText(img,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(img,(startX,startY),(endX,endY),color,2)
        
cv2.imshow("OUTPUT",img)
cv2.waitKey(0)
cv2.destroyAllWindows()
        